In [ ]:
# Import libraries

import shap
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display
from scipy.spatial.distance import cdist
from statsmodels.api import add_constant, OLS
from sklearn.ensemble import RandomForestRegressor

___
### Auxiliary functions for data processing

In [ ]:
def multivariate_outliers(df, cols, cutoff=3):
    """
    Removes outliers detected across multiple dimensions based on their
    Mahalanobis distance from the sample's multivariate mean.

    The input dataframe must have a monotonic index starting from zero.

    :param df: dataframe with input data
    :param cutoff: outlier cutoff threshold
    :return: dataframe multivariate outlier rows removed
    """
    # features' mean vector and inverse covariance matrix
    mean = df[cols].mean().values.reshape(1, -1)
    vi = np.linalg.inv(df[cols].cov())

    # estimate the Mahalanobis distance for every sample point
    md = cdist(mean, df[cols], 'mahalanobis', VI=vi)

    # filter each row by MD and remove it if it's past the threshold value
    df = pd.concat([df, pd.DataFrame(md.T, columns=['md'])], axis=1)
    df = df[df['md'] <= cutoff].drop(columns=['md'])

    return df


def corr_heatmap(df1, df2):
    """
    Plots the feature correlation heatmaps with/without outliers.

    :param df1: dataframe with input data without outliers
    :param df2: dataframe with input data with outliers
    :return: None
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

    # create heatmap without outliers
    sns.set(font_scale=1.1)
    h1 = sns.heatmap(round(df1.corr(), 2), vmin=-1, ax=ax1,
                     vmax=1, annot=True, cmap='viridis')
    h1.set_title('Feature Correlations',
                 fontdict={'fontsize': 15}, pad=12)
    h1.set_xticklabels(h1.get_xticklabels(), rotation=45)

    # create heatmap with outliers
    h2 = sns.heatmap(round(df2.corr(), 2), vmin=-1, ax=ax2,
                     vmax=1, annot=True, cmap='viridis')
    h2.set_title('Feature Correlations (With Outliers)',
                 fontdict={'fontsize': 15}, pad=12)
    h2.set_xticklabels(h2.get_xticklabels(), rotation=45)
    fig.tight_layout()
    fig.show()

    return


def scatter_plots(df1, df2):
    """
    Scatter plots of key features and price with/without outliers.

    :param df1: dataframe with input data without outliers
    :param df2: dataframe with input data with outliers
    :return: None
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

    # create plot without outliers
    h1 = ax1.scatter(x=df1['log(mileage)'], y=df1['year'],
                     c=df1['log(price)'], cmap='turbo')
    ax1.set_xlabel('Log10(Mileage)', fontsize=22)
    ax1.set_ylabel('Year', fontsize=22)
    ax1.set_title('Car Prices', fontdict={'fontsize': 25}, pad=12)
    ax1.set_yticks(np.arange(1940, 2030, 10), fontsize=20)
    ax1.set_xticks(np.arange(0, 7), fontsize=20)
    fig.colorbar(h1, ax=ax1, orientation='vertical', label='Log10(Price)')

    # create plot that includes outliers
    h2 = ax2.scatter(x=df2['log(mileage)'], y=df2['year'],
                     c=df2['log(price)'], cmap='turbo')
    ax2.set_xlabel('Log10(Mileage)', fontsize=22)
    ax2.set_ylabel('Year', fontsize=22)
    ax2.set_title('Car Prices (With Outliers)',
                  fontdict={'fontsize': 25}, pad=12)
    ax2.set_yticks(np.arange(1940, 2030, 10), fontsize=20)
    ax2.set_xticks(np.arange(0, 7), fontsize=20)
    fig.colorbar(h2, ax=ax2, orientation='vertical', label='Log10(Price)')
    fig.tight_layout()
    fig.show()

    # create scatter plot with car types
    markers = ['o', '^', 's', 'D', 'x', 'p']
    sns.lmplot(x='log(mileage)', y='year', data=df1, hue='aggcartype',
               fit_reg=False, legend=False, markers=markers, palette="Set1",
               height=9, aspect=1.1)
    plt.xlabel('Log10(Mileage)', fontsize=22)
    plt.ylabel('Year', fontsize=22)
    plt.title('Car Types', fontdict={'fontsize': 25}, pad=12)
    plt.legend(title='Car Type', fontsize=18,
               loc='lower left', markerscale=2)
    plt.rcParams['legend.title_fontsize'] = 22
    plt.yticks(np.arange(1980, 2025, 5), fontsize=20)
    plt.xticks(fontsize=20)
    plt.xlim(1, 5.5)
    plt.show()

    return

___
### Data parsing and processing

The dataset includes only a few cars with a time series of updates in the platform. This indicates that it is a cross-sectional dataset and the limited time series components from a minority of cars are removed. Only the most recent record is maintained for these cars. The car types are aggregated to broader categories that have a more uniform distribution compared to the raw data. The aggregation also provides for parsimonious models with fewer dummy variables for this feature.

The main challenge with identifying good opportunities is to disambiguate what data points consitute outliers and which ones don't. For instance, a 20-year-old car could have its engine changed after service, implying that it would have a very low mileage. In addition, an antique car could be very expensive as a luxury item. I apply a first round of sanity checks on prices, mileage and engine sizes to trim extreme outliers with unrealistic values (~5000 cars). However, this barely helps in addressing the main challenge. Even more conservative statistical criteria with univariate measures such as the Median Absolute Deviation are inadequate in removing all the outliers.

Car buyers usually consider a multitude of features when looking for a good deal. For instance, a price by itself is uninformative about the quality of the deal unless it is assessed in parallel with the car's year, mileage, etc. Univariate outlier measures fail for the same reason for this dataset, because the data is fraught with multivariate outliers (see also the comparative scatter plots with and without the outliers). I apply a multivariate measure based on the Mahalanobis Distance (MD) of each data point from the multi-dimensional sample mean. The features I consider include price, mileage and year.

The MD metric is the multivariate extension to the univariate Z-score metric for outlier treatment. As such, it is subject to the same robustness concerns as the Z-score. MD relies on sample means and the covariance matrix of the features. Both of these can be affected substantially by outliers, especially in non-normal data that exhibit nonlinear relationships (notice the L-shaped pattern in scatter plots). However, there is no simple multivariate extension of univariate robust measures such as MAD.

The MD criteria across price, mileage and year remove roughly 1500 cars, a relatively small loss compared to the remaining 15000 on the platform. However, the improvement in the quality of the sample is substantial. The cross-correlations among these features increase and many of the seemingly good deals (near the top-left quadrant in the scatter plot, low mileage for small car age) are dropped as spurious. A large part of the L-shaped nonlinearity in the scatter plots is also mitigated, although not eradicated completely.

The raw data have a very particular structure and the outlier criteria have mitigated some of its irregularities. The criteria cannot be too conservative, otherwise most points would be treated as outliers. It is best practice to have less stringent criteria and use modeling tools that have a degree of robustness to outliers. For instance, random forests might be more suitable for this amount of features and data structure compare to boosted trees. Random forests tend to be robust to outliers because the algorithm ensembles across multiple weak learners. The rationale is that the outliers will give a few bad estimators whose effect will be dimished during ensembling. Contrast to boosted trees that put larger weights on the residual of the weakest estimators in the previous round. The weakest estimators are expected to be those that are affected by outliers, implying that the algorithm assigns larger weight to the outliers compared to the rest of the sample.



In [ ]:
# read dataset and define year feature
df = pd.read_csv("listing_cars.csv").drop(columns=['Unnamed: 0'])
df['year'] = df['name'].str.lstrip().str[:5].astype(int)

# convert date strings to datetime
df['lastupdated'] = pd.to_datetime(df['lastupdated'])

# sanity checks on numeric values, truncate outliers
# with intuitive hardwired thresholds
df = df[df['usd_price'] >= 1e3]
df = df[df['enginesize'].between(800, 10000)]
df = df[df['km_mileage'] < 1e6]

# mitigate scale effects by log-transforms
scols = ['usd_price', 'km_mileage', 'enginesize']
df[['log(price)', 'log(mileage)', 'log(enginesize)']] = np.log10(df[scols])

# aggregate car types to broader categories
suv = ['SUV/Crossover', 'AWD/4WD']
luxury = ['Luxury', 'Convertible', 'Hybrid/Electric']
cycle_truck = ['Van/Minivan', 'Wagon/Touring/Estate', 'Motorcycle', 'Truck']
df['aggcartype'] = np.where(
    df['cartype'].isin(suv), 'SUV', np.where(
        df['cartype'].isin(luxury), 'Luxury', np.where(
            df['cartype'].isin(cycle_truck), 'Cycle/Truck', df['cartype'])))

# sort the data by car and date
cols = [col for col in df.columns.tolist()
        if col not in ['lastupdated', 'status']]
df = df.sort_values(by=cols + ['lastupdated'])

# # inspect duplicates, i.e. group by car with >1 rows
# dft = df[df.duplicated(subset=cols, keep=False)]

# keep only the most recently updated row for each car
df = df.drop_duplicates(subset=cols, keep='last')

# clean up some unnecessary columns
rcols = ['cartype', 'exactmodel', 'seats', 'bodyinteriorcolour',
         'registrationexpiry', 'lastserviced', 'model',
         'usd_price', 'km_mileage', 'enginesize']
df = df.drop(columns=rcols).reset_index(drop=True)

# remove multivariate outliers
cols = ['log(price)', 'year', 'log(mileage)', 'log(enginesize)']
df_outliers = df.copy()
df = multivariate_outliers(df, cols=cols, cutoff=3)

# summary statistics (numeric and some categorical features)
prc = [0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99]
s = 'Summary Statistics from a Subset of {} Features'
print('\033[1m' + s.format('Numeric') + '\033[0m')
summ = df.describe(percentiles=prc)
summ.loc['missing'] = df[summ.columns].isna().sum()
summ.loc['NaN/tot (%)'] = round(summ.loc['missing'] /
                                summ.loc['count'] * 100, 2)
display(summ)

print('\033[1m' + s.format('Categorical') + '\033[0m')
idxs = ['Automatic', 'Manual', 'Coupe', 'Cycle/Truck',
        'Hatchback', 'Luxury', 'SUV', 'Sedan']
display(df[['aggcartype', 'transmission']]
        .apply(pd.value_counts).reindex(idxs)
        .replace(np.nan, 0).astype(int)
        .replace(0, '-----', regex=True))

# plot the correlation heatmaps and scatter plots
corr_heatmap(df, df_outliers)
scatter_plots(df, df_outliers)

___
### Auxiliary functions for identifying good opportunities

In [ ]:
def random_forest_model(df_train, df_test):
    """
    Performs a random forest regression of the log-price on various features.

    :param df_train: training dataframe
    :param df_test: testing dataframe
    :return: testing dataframe with log-price predictions from the model
    """
    # setup the feature names
    dummies = [col for col in df.columns if
               col.startswith('type_') or col.startswith('gear_')]
    xcols = ['log(mileage)', 'year', 'previousowners',
             'log(enginesize)', 'ord_brand'] + dummies
    xycols = xcols + ['log(price)']

    # remove missing values
    df_train = df_train.dropna(subset=xycols)
    df_test = df_test.dropna(subset=xycols)

    # load the random forest regressor class
    rf = RandomForestRegressor(n_estimators=15, max_depth=3, random_state=0)

    # fit the model in the training set
    rf.fit(X=df_train[xcols], y=df_train['log(price)'])

    # get the model's log-price predictions in the testing set
    yhat = rf.predict(df_test[xcols])

    # estimate OOS R-squared by OLS regression between observed
    # and predicted values in the testing set, the intercept
    # enforces the metric within [0, 1]
    R2 = OLS(df_test['log(price)'], add_constant(yhat),
             missing='drop').fit().rsquared
    print('\n\033[1m' + 'Random Forest OOS Model Performance R2 = {}'
          .format(R2.round(2)) + '\033[0m')

    # place the predicted duration in the testing dataframe
    df_test['yhat'] = yhat

    # visualize feature importance results with Shapley values
    explainer = shap.TreeExplainer(rf)
    shap_values = explainer.shap_values(df_test[xcols])
    shap.summary_plot(shap_values, df_test[xcols], plot_type="bar")
    plt.show()

    return df_test


def ranking_output(df):
    """
    Summarizes the good/bad car opportunities and their ranking.

    :param df: dataframe with model predictions
    :return: None
    """
    # clean up
    cols = [col for col in df.columns if
            col.startswith('type_') or col.startswith('gear_')] + ['ord_brand']
    df = df.drop(columns=cols)

    # separate the good from the bad opportunities
    df['y-yhat'] = df['log(price)'] - df['yhat']
    df_good = df[df['y-yhat'] <= 0].sort_values(by='y-yhat', ascending=False)
    df_bad = df[df['y-yhat'] > 0].sort_values(by='y-yhat', ascending=False)

    # create rank column
    df_good['ranking'] = df_good['y-yhat'].rank(method='first').astype(int)
    df_bad['ranking'] = df_bad['y-yhat'].rank(method='first').astype(int)

    # change the order of columns in the output frames
    cols = ['ranking', 'y-yhat'] + [col for col in df.columns
                                    if col not in ['ranking', 'y-yhat']]

    # sort the output dataframes by rank
    df_good = df_good[cols].sort_values(by='ranking')
    df_bad = df_bad[cols].sort_values(by='ranking')

    # show the good/bad opportunities in descending order
    print('\n\033[1m' + 'Good Opportunities (Underpriced Cars): ' +
          'ranked best to least good' + '\033[0m')
    display(df_good)
    print('\n\033[1m' + 'Bad Opportunities (Overpriced Cars): ' +
          'ranked least bad to worst' + '\033[0m')
    display(df_bad)

___
### Finding good opportunities

The train/test data split is straighforward for this data. Sold cars are used for training the model and its predictions are tested against the Active cars on the platform. According to the model, a good opportunity arises when the marketed price is smaller than the model's prediction for this car, while a bad opportunity will have a marketed price that is higher than the predicted one. In particular, if $y$ and $\hat y$ are the marketed and predicted prices respectively, then
1. $y$ - $\hat y$ < 0 => good opportunity (car is underpriced)
2. $y$ - $\hat y$ > 0 => bad opportunity (car is overpriced)

The list of features includes log-mileage, year, an aggregated car type, brand, transmission details, number of previous owners and log-engine size. Car type and transmission have small numbers of possible values and they are encoded as dummy variables. On the contrary, brand takes too many possible values for one-hot encoding, and it is transformed into frequency-based ordinal IDs. The random forest model's feature analysis demonstrates that the only relevant features for predicting car prices are mileage, year and engine size. The relationship perhaps wouldn't have been as clear if the multivariate outliers across these three features and log-price hadn't been removed.

The output is split into two subsets. The first included the good opportunities ordered from best to worst. The second includes the overpriced cars ordered from least bad to worst possible opportunity. The ranking is directly related to the value of the difference $y$ - $\hat y$. For the good deals, the more negative this difference is, the more underpriced the car, and vice versa for the bad deals.

In [ ]:
# create car type and transmission dummies
df[['type', 'gear']] = df[['aggcartype', 'transmission']]
df = pd.get_dummies(df, columns=['type', 'gear'], drop_first=True, dtype=int)

# define ordinal brand IDs from their frequency
df['ord_brand'] = df.groupby('brand')['brand'].transform('count')

# split main data into training and testing subsets
# (save a copy of the training test for household finance question)
df_train, df_test = df[df['status'] == 'Sold'], df[df['status'] == 'Active']
df_fin = df_train.copy()

# fill in the model-predicted prices in the testing dataset
dfp = random_forest_model(df_train, df_test)

# show results
ranking_output(dfp)

___
### Weekly signal on household finances

A likely indicator variable for the state of household finances (and perhaps the consumer economy by extension) is the number of weekly car sales. Its histogram reveals broad variation across weeks, along with some clustering as well. This analysis focuses on the training dataset that contains confirmed sales, unlike marketed prices in the training set.

Candidate signals in the training dataset include the cross-sectional averages and standard deviations of sale prices and the features that were shown predictability characteristics in the model before (log-mileage, log-engine size and year of the car make. Following a grid search for the best performing combination of these features, the proposed feature is the ratio $\sigma_{km}/\sigma_{eng}$, where $\sigma_{km}$ and $\sigma_{eng}$ are the standard deviations of the log-mileage and log-engine size respectively. The coefficient of total weekly sales on weekly-lagged values of the signal is positive.

The signal implies that an increase in car mileage variance and a decrease of engine size variance predict a surge in car sales in the following week, and by extension imply an improved state of household finances. The intuition of the signal's predictability is the following:
1. An increase in the dispersion of mileage $\sigma_{km}$ among sold cars implies that the buyers' interest is spread across used and new cars. The new cars tend to be more expensive, implying that households can afford to buy such cars.
2. A simultaneous decrease in engine size dispersion $\sigma_{eng}$ implies an equilibrium of supply with a demand side (the buyers) that targets specific car specification. In other words, the buyers know what they want and are willing to purchase it, rather than exploring alternative car specifications within a lower budget.
3. The combination of these two economic forces among the buyers (demand for new and expensive cars with targeted specifications) could indicate that households are strong enough to result in a surge in sold cars.

In [ ]:
# aggregate dates into weekly intervals throughout the year
df_fin['Week'] = df_fin['lastupdated'].dt.isocalendar().week

# weekly car sales proxy the state of household finances
wk_sales = df_fin.groupby(df_fin['Week'])['name'].count()
wk_sales.plot(kind='bar', legend=False, figsize=(10, 8),
              xticks=range(0, 53, 2), ylabel='Total Weekly Sales',
              title='Weekly Sales Distribution', rot=0, fontsize=15)

In [ ]:
# grid of candidate signals
wk_avg_sale_price = df_fin.groupby(df_fin['Week'])['log(price)'].mean()
wk_avg_mileage = df_fin.groupby(df_fin['Week'])['log(mileage)'].mean()
wk_avg_engine = df_fin.groupby(df_fin['Week'])['log(enginesize)'].mean()
wk_avg_year = df_fin.groupby(df_fin['Week'])['year'].mean()
wk_std_sale_price = df_fin.groupby(df_fin['Week'])['log(price)'].std()
wk_std_mileage = df_fin.groupby(df_fin['Week'])['log(mileage)'].std()
wk_std_engine = df_fin.groupby(df_fin['Week'])['log(enginesize)'].std()
wk_std_year = df_fin.groupby(df_fin['Week'])['year'].std()

# best performing signal
signal = (wk_std_mileage / wk_std_engine).rename('sigma(mileage)/sigma(engine)')
res = OLS(wk_sales, add_constant(signal.shift()), missing='drop').fit()
print('\n\033[1m' + 'OOS performance for best-performing signal R2 = {}'
      .format(res.rsquared.round(4)) + '\033[0m')
print('\n\033[1m' + 'Model parameters' + '\033[0m')
res.params